In [237]:
!pip install -q pyomo

In [238]:
from pyomo.environ import *

In [239]:
import numpy as np

In [240]:
# create a model
model = ConcreteModel()

In [241]:
M=4
N=10

In [242]:
obj_coef_c1 = np.array([35,50,58,60,44,39,45,55,35,40])
obj_coef_c2 = np.array([72, 95, 110, 125 ,88 ,74, 95 ,115 ,60 ,84])


In [243]:
constr_coef_A= np.array(([0.2,0.2,0.25,0.15,0.1,0.25,-0.15,0.2,0.05,-0.2],[0.35,0.3,0.3,0.15,0.1,0.1,-0.05,-0.1,-0.15,-0.15],[0.15,0.1,0.05,0,0,-0.15,0.1,-0.3,-0.2,0.05],[1,1,1,1,1,1,1,1,1,1]))

In [244]:
constr_rhs_b = np.array([0,0,0,450])

In [245]:
lower_bound_x= np.array([0,0,0,0,0,0,0,0,0,0])
upper_bound_x = np.array([12, 9, 16, 25, 4 ,5 ,10, 13, 0, 6])
lower_bound_y = np.array([0,0,0,0,0,0,0,0,0,0])
upper_bound_y = np.array([450,450,450,450,450,450,450,450,450,450])

In [246]:
col_indices = np.arange(N)
print(col_indices)
print(col_indices[3])
row_indices=np.arange(M)
print(row_indices)

[0 1 2 3 4 5 6 7 8 9]
3
[0 1 2 3]


In [247]:
model.x = Var(col_indices)
model.y = Var(col_indices)

In [248]:
model.constraints = ConstraintList()

In [249]:
model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals
          5 :  None :  None :  None : False :  True :  Reals
          6 :  No

In [250]:
for i in row_indices :
     model.constraints.add(sum(constr_coef_A[i,j]*(model.x[j]+model.y[j]) for j in col_indices) == constr_rhs_b[i])

In [251]:
for j in col_indices:
   model.x[j].setlb(lower_bound_x[j])
   model.x[j].setub(upper_bound_x[j])
   model.y[j].setlb(lower_bound_y[j])
   model.y[j].setub(upper_bound_y[j])

In [252]:
model.objective = Objective(expr = sum(obj_coef_c1[j]*model.x[j]+obj_coef_c2[j]*model.y[j] for j in col_indices), sense=minimize)

In [253]:
model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    12 : False :  True :  Reals
          1 :     0 :  None :     9 : False :  True :  Reals
          2 :     0 :  None :    16 : False :  True :  Reals
          3 :     0 :  None :    25 : False :  True :  Reals
          4 :     0 :  None :     4 : False :  True :  Reals
          5 :     0 :  None :     5 : False :  True :  Reals
          6 

In [254]:
!apt-get install -y -qq glpk-utils

In [255]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 31013.4
  Upper bound: 31013.4
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 21
  Number of nonzeros: 77
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012421607971191406
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

In [256]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
  print('y[',i,'] = ', model.y[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  31013.4

Decision Variables
x[ 0 ] =  12.0
y[ 0 ] =  79.0
x[ 1 ] =  9.0
y[ 1 ] =  0.0
x[ 2 ] =  16.0
y[ 2 ] =  0.0
x[ 3 ] =  25.0
y[ 3 ] =  0.0
x[ 4 ] =  4.0
y[ 4 ] =  0.0
x[ 5 ] =  5.0
y[ 5 ] =  0.0
x[ 6 ] =  10.0
y[ 6 ] =  0.0
x[ 7 ] =  0.0
y[ 7 ] =  0.0
x[ 8 ] =  0.0
y[ 8 ] =  120.4
x[ 9 ] =  6.0
y[ 9 ] =  163.6

Constraints
constraints : Size=4
    Key : Lower : Body                   : Upper
      1 :   0.0 :                    0.0 :   0.0
      2 :   0.0 : -3.552713678800501e-15 :   0.0
      3 :   0.0 :                    0.0 :   0.0
      4 : 450.0 :                  450.0 : 450.0
